# Start
The script only works if MONTH_ACT is in H1, because it computes MONTH_PREV from the last year only

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import akshare as ak  # https://github.com/akfamily/akshare
pd.set_option('display.max_rows', 500)

%matplotlib inline

In [2]:
def _parse_quarter(s):
    x = s.dayofyear
    if x <= 90:
        q = 'Q1'
    elif x <= 181:
        q = 'Q2'
    elif x <= 273:
        q = 'Q3'
    else:
        q = 'Q4'
    return str(s.year) + q
def get_quarter(df, date_column='date'):
    df['DATE'] = pd.to_datetime(df[date_column])
    df['dayofyear'] = df['DATE'].dt.dayofyear
    df.loc[df['DATE'].dt.is_leap_year, 'dayofyear'] -= 1
    df['year'] = df['DATE'].dt.year
    df['quarter'] = df.apply(_parse_quarter, axis=1)
    df = df.set_index('quarter')
    df = df.sort_index()
    return df

def pick_stocks(df, cash_to_invest):
    '''Pick stocks based on momentum and pb, used in the magic formula
    - Look for the TOP_BY_MMT fraction of companies that have the largest MMT_VAR
    - Sort these companies by price-to-book value in ascending order 
        and take the top N_STOCKS stocks
    Returns
        A df with all the stocks chosen and their MMT_VAR and PB
    Note:
        MMT_VAR is a momentum variable like 6 month price momentum
    '''

    cash_for_each_stock = cash_to_invest / N_STOCKS
    top_by_mmt = df[
            (~df['book_value_per_share'].isnull()) & 
            (df['price_to_book_ratio'] > 0)
        ].sort_values(MMT_VAR, ascending=False).iloc[:round(len(df) * TOP_BY_MMT), :]
    stocks_by_mmt_pb = top_by_mmt.sort_values('price_to_book_ratio').head(N_STOCKS).copy()
    stocks_by_mmt_pb['shares_bought'] = cash_for_each_stock / stocks_by_mmt_pb['stock_price']
    stocks_by_mmt_pb_simple = stocks_by_mmt_pb[['stock', 'shares_bought', 'stock_price', 
                                                MMT_VAR, 'price_to_book_ratio']]
    stocks_by_mmt_pb_simple = stocks_by_mmt_pb_simple\
        .rename({'stock_price': 'stock_price_bought',
                 MMT_VAR: MMT_VAR + '_bought',
                 'price_to_book_ratio': 'price_to_book_ratio_bought'}, axis=1)
    return stocks_by_mmt_pb_simple


## Parameters

In [174]:
# Which stock was split in the past year; check back after finding those anomalous momentum
STOCK_SPLIT = ['IDONTKNOW']

## When to buy and sell
MONTH_ACT = 2  
YEAR_ACT = 2023
QUARTER_ACT = f'Q{MONTH_ACT // 3 + 1}'  # Dummy variable. If acting on Feb, the quarter is Q1

# Starting month to compute 6m momentum
MONTH_PREV = MONTH_ACT - 6 if MONTH_ACT - 6 > 0 else MONTH_ACT + 6  
# Starting year to compute 6m momentum
YEAR_PREV = YEAR_ACT if MONTH_ACT - 6 > 0 else YEAR_ACT - 1
# The quarter for the BV. Since the ER date differs, this should be 2 quarters ahead of QUARTER_ACT
# so that the BV is available for every stock. If acting in Q1, use previous Q3 
QUARTER_BV = f'Q{MONTH_ACT // 3 + 3}'  
# Trade on which trading day of the month. 
# The maximum of DOM varies by month; so be careful using a big value (> 20)
# Previously I use the first day of the MONTH_ACT for convienience. Now using DOM_TRADING allows me to
# compute on any day of the month. Note that the starting and end dates of 6m momentum is defined as 
# the Nth trading day of MONTH_PREV and that of MONTH_ACT, where N = DOM_TRADING. Therefore they do not
# always fall on the same calendar day of month
DOM_TRADING = 3  
DATE_CUTOFF = f"{YEAR_ACT}-{f'{MONTH_ACT}'.zfill(2)}-01"  # To filter data without recent data

## Filters to pick stocks and compute gains
N_TOP_BY_MKT_CAP = 300  # Choose from the top N of sp500
TOP_BY_MMT = 0.2  # The top fraction of stocks ranked by MMT
MMT_VAR = 'stock_price_mmt_6m'
TAX_FACTOR = (1 - 0.1)
N_STOCKS = 40  # Number of stocks to buy
TOTAL_CASH = 100000.0  # Total cash

## Used mainly for test, so relax it for prediction
MIN_QUARTERS = 2  # Remove stocks with fewer than some quarters
MAX_QUARTERS_NEG_PB = 40  # Remove stocks with more than some quarters with negatives pb

# Download SP500 component history
Downloading the top 500 takes about xxx hours

In [42]:
## Get fundamental data (seasonal) or price data (daily) 
# Download all SP500 instead of N_TOP_BY_MKT_CAP, since later filters will remove some

is_from_scratch = False  # If starting from scratch and no stock data has been downloaded already
to_download = 'fundamental'  # 'fundamental' or 'price', download fundamental or price data

# Get stock list (ordered by capital)
df_sp500_list = pd.read_excel('sp500_fulllist_ranked.xlsx', engine='openpyxl', sheet_name=str(YEAR_ACT))
stocks = df_sp500_list.loc[df_sp500_list['stock'] != 'GOOG', 'stock'].values
stocks += ['OHI']  # I like OHI

file_name = {
    'fundamental': f'sp500_history_raw_{str(YEAR_ACT)}.csv',
    'price': f'sp500_history_price_raw_{str(YEAR_ACT)}.csv',
}
min_row = {
    'fundamental': 3,
    'price': 190,
}
anom = {'Failed_PB': [], 'Short': [], 'Failed_P', []}

if not is_from_scratch: 
    print('Read downloaded stocks from local file')
    df_stock_all = pd.read_csv(file_name[to_download])

# If df_stock_all does not ecist, declare it
try:
    df_stock_all       
except NameError: 
    df_stock_all = pd.DataFrame()
stock_downloaded = [] if df_stock_all.empty else df_stock_all.stock.unique()
print(f'Downloaded {len(stock_downloaded)}, {stock_downloaded}')
    
for stock_symbol in stocks:
    if stock_symbol in stock_downloaded:
        continue
    try:
        if to_download == 'fundamental':  # Download PB and PE
            try:  # If PE data can't be downloaded, record the stock
                df_pb = ak.stock_us_fundamental(stock=stock_symbol, symbol="PB")
            except ValueError:
                print(f'Download of {stock_symbol} PB failed. Skip for now')
                anom['Failed_PB'].append(stock_symbol)
                continue
            try:  # If PE data can't be downloaded, create and make it all nan
                df_pe = ak.stock_us_fundamental(stock=stock_symbol, symbol="PE")
            except ValueError:
                print(f'Download of {stock_symbol} PE failed. Using nan instead')
                df_pe = df_pb.copy()
                df_pe.columns = ['date', 'stock_price', 'ttm_net_eps', 'pe_ratio']
                df_pe[['stock_price', 'ttm_net_eps', 'pe_ratio']] = np.nan
            df_stock = pd.merge(df_pe.drop('stock_price', axis=1), df_pb, on='date')
        elif to_download == 'price':  # Download price
            df_stock = ak.stock_us_daily(symbol=stock_symbol)
            df_stock = df_stock.reset_index()  # Set the date index to a column
        else:
            print('Wrong variable name')
    except IndexError:
        print(f'Failed for {stock_symbol}')
        anom['Failed_P'].append(stock_symbol)
        continue        
    df_stock['stock'] = stock_symbol
    df_stock_all = df_stock_all.append(df_stock)
    print(f"{(stock_symbol, df_stock.date.min(), df_stock.date.max(), len(df_stock))}")
    if len(df_stock) < min_row[to_download]:
        anom['Short'].append(stock_symbol)
    df_stock_all.to_csv(file_name[to_download], index=False)    

Read downloaded stocks from local file
Downloaded 142, ['AAPL' 'MSFT' 'AMZN' 'GOOGL' 'BRK.B' 'NVDA' 'XOM' 'UNH' 'JNJ' 'JPM'
 'TSLA' 'V' 'PG' 'HD' 'MA' 'META' 'CVX' 'MRK' 'LLY' 'ABBV' 'PFE' 'BAC'
 'AVGO' 'KO' 'PEP' 'TMO' 'COST' 'WMT' 'MCD' 'CSCO' 'ABT' 'DIS' 'DHR' 'ACN'
 'CMCSA' 'VZ' 'WFC' 'ADBE' 'NEE' 'LIN' 'NFLX' 'TXN' 'NKE' 'PM' 'CRM' 'BMY'
 'COP' 'RTX' 'QCOM' 'HON' 'AMGN' 'ORCL' 'T' 'CAT' 'UPS' 'LOW' 'IBM' 'MS'
 'UNP' 'INTC' 'SPGI' 'SBUX' 'SCHW' 'AMD' 'BA' 'GS' 'PLD' 'ELV' 'DE' 'INTU'
 'CVS' 'BLK' 'MDT' 'GILD' 'LMT' 'AMT' 'C' 'ADP' 'AMAT' 'CB' 'TJX' 'CI'
 'BKNG' 'AXP' 'ISRG' 'PYPL' 'NOW' 'MDLZ' 'GE' 'ADI' 'TMUS' 'MMC' 'SYK'
 'VRTX' 'MO' 'SLB' 'DUK' 'EOG' 'REGN' 'ZTS' 'PGR' 'TGT' 'SO' 'BDX' 'APD'
 'AON' 'CSX' 'MU' 'EQIX' 'LRCX' 'NOC' 'FISV' 'BSX' 'MRNA' 'ITW' 'ETN'
 'TFC' 'PNC' 'EL' 'MMM' 'FCX' 'HUM' 'CL' 'CCI' 'USB' 'CME' 'MPC' 'KLAC'
 'NSC' 'ICE' 'SHW' 'WM' 'PXD' 'VLO' 'HCA' 'MCK' 'ATVI' 'SNPS' 'DG' 'EMR'
 'GD' 'D']
Download of GM PB failed. Skip for now
('PSX', '2013-03-31', '2023

('CHD', '2009-12-31', '2023-01-31', 53)
('CTRA', '2009-06-30', '2022-09-30', 52)
('WAT', '2009-12-31', '2022-09-30', 52)
('MKC', '2009-11-30', '2022-11-30', 53)
('CAH', '2009-12-31', '2023-01-30', 53)
('NTRS', '2009-12-31', '2023-01-30', 53)
('WAB', '2009-12-31', '2022-09-30', 52)
('TSN', '2009-12-31', '2023-01-31', 53)
('EPAM', '2012-12-31', '2023-01-30', 41)
('TDY', '2009-12-31', '2023-01-30', 53)
('WST', '2009-12-31', '2022-09-30', 52)
('CNP', '2009-12-31', '2023-01-31', 53)
('MPWR', '2009-12-31', '2022-09-30', 52)
('INVH', '2017-12-31', '2022-09-30', 20)
('XYL', '2012-09-30', '2022-09-30', 41)
('MAA', '2009-12-31', '2022-09-30', 52)
('ALGN', '2009-12-31', '2022-09-30', 52)
('LVS', '2009-12-31', '2022-09-30', 52)
('STLD', '2009-12-31', '2023-01-31', 53)
('BALL', '2009-12-31', '2023-01-31', 53)
('DRI', '2009-11-30', '2022-11-30', 53)
('AES', '2009-12-31', '2023-01-31', 53)
('CMS', '2009-12-31', '2023-01-31', 53)
('MRO', '2009-12-31', '2023-01-31', 53)
('BR', '2009-12-31', '2023-01-30

# Apply the magic-formula for a certain year

**Caveat**
- In my test (Stock_MA.ipynb), I use the first trading day of MONTH_ACT and first trading day of MONTH_PREV to define the the two ends of the 6m momentum. Here I use the Nth trading day instead. Therefore the result may vary
- If stock splits, the price momentum is unreasonable, I will check it and likely skip that stockthat year
- If using finviz
 - The BV is updated with Q4 results for some stocks
 - I can only use the whole SP500 instead of top 300
 - The momentum filter is not by rank, but by value like 10% increase 

**Methodology (for test)** 

This is from the backtest. Prediction is similar except I only use one year's data and optimized parameters


Here I get daily price history data and use the price of the action day (The Nth trading day of the action month) to compute price momentum. Also I use a latest full list of SP500, but take only the first N stocks

Method: 
- Select the N_TOP_BY_MKT_CAP top stocks from SP500
- Starting from 2010, take actions on the Nth trading day of Feb(MONTH_ACT) of each year
- First choose the top 20% (TOP_BY_MMT = 0.2) of stocks ranked by 6 month price momentum 
(price of the Nth trading day of MONTH_ACT minus price of the Nth trading day of previous MONTH_PREV)
- Then choose the top N_STOCKS ranked by PB (price of first day of MONTH_ACT / book value of previous Q3 or 2 quarters ago)
- On the Nth trading day of MONTH_ACT of each year, sell all stocks from the previous year with tax rate of 90% (TAX_FACTOR = (1 - 0.1)), and buy new stocks with the money by repeating the previous two steps  

Results:
- Act on the first trading day of Feb, using price of that day and BV of Q3
- Choose the top 40 gives good return
- Each year, the gain (after 10% lt tax) is better than SPY

## Processe PB

### Filter by available length and action quarter

In [97]:
df_stock_all = pd.read_csv(file_name['fundamental'])
print(f'{df_stock_all.stock.nunique()} stocks downloaded')

# Remove data with too short length
df_stock_sub = df_stock_all[df_stock_all.stock.isin((df_stock_all.groupby('stock').size() >= 
                                                     MIN_QUARTERS).index.values)].copy()
df_stock_sub = get_quarter(df_stock_sub)
df_stock_sub = df_stock_sub.drop(['DATE', 'dayofyear', ], axis=1).sort_values(['date', 'stock'])
# Convert $dollar to float
df_stock_sub['ttm_net_eps'] = df_stock_sub.ttm_net_eps.str[1:].replace('', np.nan).astype(float)
df_stock_sub['book_value_per_share'] = df_stock_sub.book_value_per_share.str[1:].replace('', np.nan).\
    str.replace(',', '').astype(float)
# Replace inf pe or pb to 0
df_stock_sub = df_stock_sub.replace(np.inf, 0)
# Remove stocks with over XX quarters with neg equity (pb)
df_stock_sub_neg = df_stock_sub[df_stock_sub.price_to_book_ratio < 0].groupby('stock').size()
stocks_sub_neg = df_stock_sub_neg[df_stock_sub_neg >= MAX_QUARTERS_NEG_PB].index.values
df_stock_sub = df_stock_sub[~df_stock_sub.stock.isin(stocks_sub_neg)]

print(f'{df_stock_sub.stock.nunique()} stocks remained after filtering by data length')
df_stock_sub.to_csv('sp500_history_filterd.csv')

498 stocks downloaded
493 stocks remained after filtering by data length


In [107]:
## Get data for the quarter needed
df_pb_history = pd.read_csv('sp500_history_filterd.csv', index_col=0)
print(f'{df_pb_history.stock.nunique()} stocks after filtering loaded')

df_pb_quarter = df_pb_history[df_pb_history.index.str.endswith(QUARTER_BV)]\
    [['date', 'stock_price', 'book_value_per_share', 'stock', 'year']]\
    .reset_index(drop=True).rename({'year': 'year_prev', 'stock_price': 'stock_price_pb'}, axis=1)
print(f'{df_pb_quarter.stock.nunique()} stocks with the right quarter')

print(f'Stocks without {QUARTER_BV} data are {set(df_pb_history.stock.unique()) - set(df_pb_quarter.stock.unique())}')
# It's a new stock

493 stocks after filtering loaded
492 stocks with the right quarter
Stocks without Q3 data are {'GEHC'}


### Manually check and fill stocks with PB failure

In [130]:
stocks_failed_pb = ['GM', 'HSY', 'CSGP', 'STT' ]  #  anom['Failed_P']
stocks_failed_pb_bv = [48.95, 15.26, 16.49, 69.7]   # get from https://www.macrotrends.net/

df_stocks_failed_pb = pd.DataFrame([
    [df_pb_quarter.date.max()] * len(stocks_failed_pb),
    [np.nan] * len(stocks_failed_pb),
    stocks_failed_pb_bv,
    stocks_failed_pb,
    [df_pb_quarter.year_prev.max()] * len(stocks_failed_pb)
]).T
df_stocks_failed_pb.columns = df_pb_quarter.columns

df_pb_quarter = df_pb_quarter.append(df_stocks_failed_pb)
print(f'{df_pb_quarter.stock.nunique()} stocks after manully filling in those failing in PB download')

## Get price momentum

In [ ]:
df_p_history = pd.read_csv('sp500_history_price_raw.csv')

In [144]:
## do a test run with fewer data
# df_p_history = df_p_history.head(100000)
# DATE_CUTOFF = '2020-01-01'

In [145]:
df_p_history['date'] = pd.to_datetime(df_p_history['date'])
## Remove stocks that didn't last until the recent
dt_cutoff = pd.to_datetime(DATE_CUTOFF)
df_p_history['max_date'] = df_p_history.groupby('stock')['date'].transform(max)
df_p_history = (df_p_history[df_p_history['max_date'] >= dt_cutoff]).drop('max_date', axis=1)

## Get the starting and end dates to compute the momentum
# The starting date is the DOM_TRAIDING day of MONTH_ACT, and the end date is 
# the DOM_TRAIDING day of MONTH_PREV. The two may not be the same calendar DOM
df_p_history = df_p_history.sort_values(['stock', 'date'])
df_p_history['year'] = df_p_history.date.dt.year
df_p_history['month'] = df_p_history.date.dt.month
df_p_history['dom_trading'] = df_p_history.groupby(['stock', 'year', 'month'])['date'].rank()
df_p_history_prev = df_p_history[(df_p_history.month == MONTH_PREV) &
                                 (df_p_history.dom_trading == DOM_TRADING)]
df_p_history_curr = df_p_history[(df_p_history.month == MONTH_ACT) &
                                 (df_p_history.dom_trading == DOM_TRADING)]
## Use year_prev to join two datasets. 
# For 6month range, if MONTH_ACT is Jul-Dec, the year (year_prev) of MONTH_PREV is 
# the same year; otherwise its the previous year
df_p_history_curr['year_prev'] = df_p_history_curr.year if MONTH_ACT - 6 > 0 \
    else df_p_history_curr.year - 1
df_p_history_prev['year_prev'] = df_p_history_prev.year
cols = ['date', 'close', 'stock', 'year_prev']
df_p_history_mmt = pd.merge(df_p_history_prev[cols], 
                            df_p_history_curr[cols + ['year']],
                            on=['stock', 'year_prev'], 
                            suffixes=['_prev', '']
                           )

## Get the momentum
df_p_history_mmt['stock_price_mmt_6m'] = df_p_history_mmt['close'] / df_p_history_mmt['close_prev'] - 1

## Merge with PB

In [171]:
df_p_pb = pd.merge(df_p_history_mmt[['stock', 'year_prev', 'date', 'close', 
                                     'year', 'stock_price_mmt_6m']], 
                   df_pb_quarter, 
                   on=['stock', 'year_prev'], 
                   suffixes=['', '_pb'])\
            .rename({'close': 'stock_price'}, axis=1)\
            .drop(['year_prev'], axis=1)

df_p_pb['price_to_book_ratio'] = df_p_pb['stock_price'] / df_p_pb['book_value_per_share']

## Get the stocks based on the magic formula

### Combined price and PB data and get the right year

In [221]:
# Choose only the top N companies of SP500 to start with
df_rank = pd.read_csv('sp500_fulllist_ranked.csv')
top_stocks = df_rank[df_rank['rank'] <= N_TOP_BY_MKT_CAP].stock.values
df_p_pb = df_p_pb[df_p_pb.stock.isin(top_stocks)]
df_row = df_p_pb[df_p_pb.year == YEAR_ACT]

### Look for anoumalous mmt to detect split

In [169]:
df_row.sort_values('stock_price_mmt_6m')

,date_prev,close_prev,stock,year_prev,date,close,year,stock_price_mmt_6m
39,2020-08-05,440.25,AAPL,2020,2021-02-03,133.9400,2021,-0.695764
279,2020-08-05,1485.02,TSLA,2020,2021-02-03,854.6900,2021,-0.424459
211,2020-08-05,328.00,MA,2020,2021-02-03,332.8200,2021,0.014695
328,2020-08-05,196.10,V,2020,2021-02-03,201.3600,2021,0.026823
63,2020-08-05,3205.03,AMZN,2020,2021-02-03,3312.5300,2021,0.033541
97,2020-08-05,249.12,FB,2020,2021-02-03,266.6500,2021,0.070368
315,2020-08-05,312.47,UNH,2020,2021-02-03,337.8900,2021,0.081352
155,2020-08-05,148.40,JNJ,2020,2021-02-03,160.5000,2021,0.081536
369,2020-08-05,129.81,WMT,2020,2021-02-03,141.2000,2021,0.087744
246,2020-08-05,212.94,MSFT,2020,2021-02-03,243.0000,2021,0.141167


In [ ]:
STOCK_SPLIT = [dasfsafda]
stocks_split = STOCK_SPLIT
df_row = df_row[~df_row.stock.isin(stocks_split)]

### Apply the magic formula to get candidate stocks

In [232]:
stocks_invested = pick_stocks(df_row, cash_to_invest=TOTAL_CASH)
stocks_invested['shares_bought'] = stocks_invested['shares_bought'].round()
stocks_invested['stock_price_mmt_6m_bought'] = (stocks_invested['stock_price_mmt_6m_bought'] * 100).round()

In [233]:
# stocks_invested.sort_values('stock_price_mmt_6m_bought', ascending=False)
cols = ['stock', 'price_to_book_ratio_bought', 'stock_price_mmt_6m_bought']
stocks_invested.sort_values('price_to_book_ratio_bought').reset_index(drop=True)

,stock,shares_bought,stock_price_bought,stock_price_mmt_6m_bought,price_to_book_ratio_bought
0,AIG,42.0,59.11,25.0,0.752419
1,MET,37.0,68.09,18.0,0.829354
2,BK,41.0,60.69,18.0,1.142292
3,WFC,45.0,55.60,21.0,1.163180
4,TFC,39.0,64.02,18.0,1.240457
5,AFL,40.0,62.89,15.0,1.242394
6,MPC,34.0,74.02,36.0,1.316379
7,ED,29.0,86.05,16.0,1.397823
8,FITB,54.0,45.94,27.0,1.407044
9,BAC,53.0,46.94,24.0,1.419843


In [ ]:
# https://finviz.com/screener.ashx?v=152&f=idx_sp500,ta_perf_26w10o&ft=3&o=pb
# https://finviz.com/screener.ashx?v=151&f=idx_sp500,ta_perf_26w20o&ft=4&o=pb